# Intro

This notebook is based on 'ImageNet training in PyTorch' [example](https://github.com/pytorch/examples/blob/master/imagenet/main.py).

The goal of this notebook to demonstrate how to use [NNCF](https://github.com/openvinotoolkit/nncf) 8-bit quantization to optimize the PyTorch model for inference with OpenVINO Toolkit. For more advanced usage refer to these [examples](https://github.com/openvinotoolkit/nncf/tree/develop/examples)

To make downloading and training fast, we use resnet-18 model with tiny-imagenet-200 dataset. 

# Settings
Set the name for the model, and the image width and height that will be used for the network. Also define paths where PyTorch, ONNX and OpenVINO IR models will be stored.

In [2]:
from pathlib import Path

MODEL_DIR = Path('model')
OUTPUT_DIR = Path('output')
BASE_MODEL_NAME = 'resnet18'
image_size = 64

OUTPUT_DIR.mkdir(exist_ok=True)
MODEL_DIR.mkdir(exist_ok=True)

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
fp32_pth_path = Path(MODEL_DIR / (BASE_MODEL_NAME + '_fp32')).with_suffix('.pth') 
fp32_onnx_path = Path(OUTPUT_DIR / (BASE_MODEL_NAME + '_fp32')).with_suffix(".onnx")
fp32_ir_path = fp32_onnx_path.with_suffix(".xml")
int8_onnx_path = Path(OUTPUT_DIR / (BASE_MODEL_NAME + '_int8')).with_suffix('.onnx') 
int8_ir_path = int8_onnx_path.with_suffix(".xml")

time: 3.85 ms (started: 2021-08-07 15:28:11 +03:00)


Download Tiny ImageNet dataset
* 100k images of shape 3x64x64
* 200 different classes: snakes, spaiders, cats, trucks, grasshopper, gull, etc.

In [3]:
def download_tiny_imagenet_200(output_dir,
                        url='http://cs231n.stanford.edu/tiny-imagenet-200.zip',
                        tarname='tiny-imagenet-200.zip'):
    from urllib.request import urlretrieve
    output_dir.mkdir(exist_ok=True)
    archive_path = output_dir / tarname
    urlretrieve(url, archive_path)
    import zipfile
    zip_ref = zipfile.ZipFile(archive_path, 'r')
    zip_ref.extractall(path=output_dir)
    zip_ref.close()
    print(f'Successfully downloaded and extracted dataset to: {output_dir}')

DATASET_DIR = OUTPUT_DIR / 'tiny-imagenet-200'
if not DATASET_DIR.exists():
    download_tiny_imagenet_200(OUTPUT_DIR)

time: 15.1 ms (started: 2021-08-07 15:28:11 +03:00)


# Install pre-requisites

Create a separate Python* virtual environment and install the following prerequisites into it:

In [4]:
!pip install nncf[torch]
!pip install openvino

time: 2.11 s (started: 2021-08-07 15:28:11 +03:00)


# Import NNCF from your Python* code

In [5]:
import sys
import time

import torch
import nncf  # Important - should be imported directly after torch
from nncf import NNCFConfig
from nncf.torch import create_compressed_model
from nncf.torch import register_default_init_args

import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device
time: 33.3 s (started: 2021-08-07 15:28:13 +03:00)


# Pre-train floating-point model
Using NNCF for model compressed implies that the user has a pre-trained model and a training pipeline to get it.

Here we demonstrate one of the possible training pipeline: the ResNet18 model pre-trained on 1000 classes of ImageNet is tuned on 200 classes of Tiny-Imagenet. 

Subsequently, the training and validation functions will be reused as is for quantization-aware training.


## Train function

In [6]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':3.3f')
    losses = AverageMeter('Loss', ':2.3f')
    top1 = AverageMeter('Acc@1', ':2.2f')
    top5 = AverageMeter('Acc@5', ':2.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, losses, top1, top5],
        prefix="Epoch:[{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        images = images.to(device)
        target = target.to(device)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do opt step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        print_frequency = 50
        if i % print_frequency == 0:
            progress.display(i)


time: 1.39 ms (started: 2021-08-07 15:28:46 +03:00)


## Validate function

In [7]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':3.3f')
    losses = AverageMeter('Loss', ':2.3f')
    top1 = AverageMeter('Acc@1', ':2.2f')
    top5 = AverageMeter('Acc@5', ':2.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            print_frequency = 10
            if i % print_frequency == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    return top1.avg

time: 13.9 ms (started: 2021-08-07 15:28:46 +03:00)


## Helpers

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

time: 8.05 ms (started: 2021-08-07 15:28:46 +03:00)


## Run training pipeline
Tune floating-point ResNet-18 on Tiny-ImageNet-200 to obtain a pre-trained model for quantization.

> **NOTE** The model is not tuned till the final accuracy. For the sake of simplicity of the tutorial, we tune for 4 epoch only and take the last model state for quantization.



In [9]:
num_classes = 200  # 200 is for tiny-imagenet, default is 1000 for imagenet
init_lr = 1e-4
batch_size = 256
epochs = 4

# create model
model = models.resnet18(pretrained=True)
# update the last FC layer for tiny-imagenet number of classes
model.fc = nn.Linear(in_features=512, out_features=num_classes, bias=True)
model.to(device)

# Data loading code
train_dir = DATASET_DIR / 'train'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [80000, 20000], 
                                                           generator=torch.Generator().manual_seed(0))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True, sampler=None)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

time: 2.57 s (started: 2021-08-07 15:28:46 +03:00)


In [10]:
acc1 = 0
best_acc1 = 0
if fp32_pth_path.exists():
    #
    # ** WARNING: torch.load functionality uses Python's pickling facilities that
    # may be used to perform arbitrary code execution during unpickling. Only load the data you
    # trust.
    #
    checkpoint = torch.load(str(fp32_pth_path), map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'], strict=True)
    acc1 = checkpoint['acc1']
else:
    # Training loop
    for epoch in range(0, epochs):
        # run a single training epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion)

        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if is_best:
            checkpoint = {'state_dict': model.state_dict(), 'acc1': acc1}
            torch.save(checkpoint, fp32_pth_path)
                
print(f'Accuracy of FP32 model: {acc1:.3f}')

Accuracy of FP32 model: 53.235
time: 29.3 ms (started: 2021-08-07 15:28:49 +03:00)


Export FP32 model to ONNX that is supported by the OpenVINO™ toolkit to benchmark it in comparison with INT8 model.

In [11]:
dummy_input = torch.randn(1, 3, image_size, image_size).to(device)
    
torch.onnx.export(model, dummy_input, fp32_onnx_path)
print(f"FP32 ONNX model was exported to {fp32_onnx_path}.")

FP32 ONNX model was exported to output/resnet18_fp32.onnx.
time: 505 ms (started: 2021-08-07 15:28:49 +03:00)


# Create and initialize quantization

NNCF enables compression-aware training by being integrated into the regular training pipelines. The framework is designed so that the modifications to your original training code are minor. 
Quantization is the simplest scenario and requires only 3 modifications.

1. Configure NNCF parameters to specify compression

In [12]:
nncf_config_dict = {
    "input_info": {
        "sample_size": [1, 3, image_size, image_size]
    },
    "log_dir": str(OUTPUT_DIR), # log directory for NNCF-specific logging outputs
    "compression": {
        "algorithm": "quantization",  # specify the algorithm here
    }
}
nncf_config = NNCFConfig.from_dict(nncf_config_dict)

time: 11.9 ms (started: 2021-08-07 15:28:49 +03:00)


2. Provide data loader to initialize the values of quantization ranges  and determine which activation should be signed or unsigned from the collected statistics using given number of samples.


In [13]:
nncf_config = register_default_init_args(nncf_config, train_loader)

INFO:nncf:Please, provide execution parameters for optimal model initialization
time: 879 ms (started: 2021-08-07 15:28:49 +03:00)


3. Create a wrapped model ready for compression fine-tuning wrom a pre-trained FP32 model and a configuration object.

In [14]:
compression_ctrl, model = create_compressed_model(model, nncf_config)

INFO:nncf:Wrapping module ResNet/Conv2d[conv1] by ResNet/NNCFConv2d[conv1]
INFO:nncf:Wrapping module ResNet/Sequential[layer1]/BasicBlock[0]/Conv2d[conv1] by ResNet/Sequential[layer1]/BasicBlock[0]/NNCFConv2d[conv1]
INFO:nncf:Wrapping module ResNet/Sequential[layer1]/BasicBlock[0]/Conv2d[conv2] by ResNet/Sequential[layer1]/BasicBlock[0]/NNCFConv2d[conv2]
INFO:nncf:Wrapping module ResNet/Sequential[layer1]/BasicBlock[1]/Conv2d[conv1] by ResNet/Sequential[layer1]/BasicBlock[1]/NNCFConv2d[conv1]
INFO:nncf:Wrapping module ResNet/Sequential[layer1]/BasicBlock[1]/Conv2d[conv2] by ResNet/Sequential[layer1]/BasicBlock[1]/NNCFConv2d[conv2]
INFO:nncf:Wrapping module ResNet/Sequential[layer2]/BasicBlock[0]/Conv2d[conv1] by ResNet/Sequential[layer2]/BasicBlock[0]/NNCFConv2d[conv1]
INFO:nncf:Wrapping module ResNet/Sequential[layer2]/BasicBlock[0]/Conv2d[conv2] by ResNet/Sequential[layer2]/BasicBlock[0]/NNCFConv2d[conv2]
INFO:nncf:Wrapping module ResNet/Sequential[layer2]/BasicBlock[0]/Sequential[do

INFO:nncf:Collecting tensor statistics ████████████████  | 1 / 1
INFO:nncf:Set sign: True and scale: [2.6400, ] for TargetType.OPERATOR_POST_HOOK /nncf_model_input_0
INFO:nncf:Performing signed activation quantization for: TargetType.OPERATOR_POST_HOOK /nncf_model_input_0
INFO:nncf:Set sign: False and scale: [3.5188, ] for TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer1]/BasicBlock[0]/ReLU[relu]/relu__0
INFO:nncf:Performing unsigned activation quantization for: TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer1]/BasicBlock[0]/ReLU[relu]/relu__0
INFO:nncf:Set sign: True and scale: [5.4515, ] for TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer1]/BasicBlock[0]/BatchNorm2d[bn2]/batch_norm_0
INFO:nncf:Performing signed activation quantization for: TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer1]/BasicBlock[0]/BatchNorm2d[bn2]/batch_norm_0
INFO:nncf:Set sign: False and scale: [2.5448, ] for TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer1]/BasicBlock[1]/ReLU[rel

INFO:nncf:Performing unsigned activation quantization for: TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer3]/BasicBlock[0]/ReLU[relu]/relu__1
INFO:nncf:Set sign: False and scale: [5.5993, ] for TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer3]/BasicBlock[1]/ReLU[relu]/relu__1
INFO:nncf:Performing unsigned activation quantization for: TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer3]/BasicBlock[1]/ReLU[relu]/relu__1
INFO:nncf:Set sign: False and scale: [3.9081, ] for TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer4]/BasicBlock[0]/ReLU[relu]/relu__1
INFO:nncf:Performing unsigned activation quantization for: TargetType.OPERATOR_POST_HOOK ResNet/Sequential[layer4]/BasicBlock[0]/ReLU[relu]/relu__1
INFO:nncf:Set sign: True and scale: [0.7697, 0.3192, 0.1000, 0.1759, 0.1000, 0.4172, 0.5834, 0.1000, 0.7551, 0.1000, ... (first 10/64 elements shown only) ] for TargetType.OPERATION_WITH_WEIGHTS ResNet/NNCFConv2d[conv1]/conv2d_0
INFO:nncf:Performing signed weight quantizati

INFO:nncf:Performing signed weight quantization for: TargetType.OPERATION_WITH_WEIGHTS ResNet/Sequential[layer4]/BasicBlock[0]/Sequential[downsample]/NNCFConv2d[0]/conv2d_0
INFO:nncf:Set sign: True and scale: [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1019, 0.1000, 0.1000, ... (first 10/512 elements shown only) ] for TargetType.OPERATION_WITH_WEIGHTS ResNet/Sequential[layer4]/BasicBlock[1]/NNCFConv2d[conv1]/conv2d_0
INFO:nncf:Performing signed weight quantization for: TargetType.OPERATION_WITH_WEIGHTS ResNet/Sequential[layer4]/BasicBlock[1]/NNCFConv2d[conv1]/conv2d_0
INFO:nncf:Set sign: True and scale: [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1317, 0.1000, 0.1000, ... (first 10/512 elements shown only) ] for TargetType.OPERATION_WITH_WEIGHTS ResNet/Sequential[layer4]/BasicBlock[1]/NNCFConv2d[conv2]/conv2d_0
INFO:nncf:Performing signed weight quantization for: TargetType.OPERATION_WITH_WEIGHTS ResNet/Sequential[layer4]/BasicBlock[1]/NNCFConv2d[conv2]/co

Evaluate the obtained model on the validation set after initialization of quantization. The accuracy should be close to the accuracy of the floating-point model in a simple case like we are demonstrating now.

In [15]:
acc1 = validate(val_loader, model, criterion)
print(f'Accuracy of initialized INT8 model: {acc1:.3f}')


Test: [ 0/79]	Time 0.325 (0.325)	Loss 2.006 (2.006)	Acc@1 55.47 (55.47)	Acc@5 80.47 (80.47)
Test: [10/79]	Time 0.046 (0.075)	Loss 1.634 (1.903)	Acc@1 59.77 (53.69)	Acc@5 82.42 (79.30)
Test: [20/79]	Time 0.041 (0.061)	Loss 1.832 (1.882)	Acc@1 55.08 (54.30)	Acc@5 80.47 (79.78)
Test: [30/79]	Time 0.037 (0.056)	Loss 1.868 (1.879)	Acc@1 55.86 (54.36)	Acc@5 76.56 (79.60)
Test: [40/79]	Time 0.052 (0.054)	Loss 1.828 (1.869)	Acc@1 53.52 (54.31)	Acc@5 80.86 (79.84)
Test: [50/79]	Time 0.066 (0.053)	Loss 1.961 (1.873)	Acc@1 50.39 (54.22)	Acc@5 82.03 (79.91)
Test: [60/79]	Time 0.037 (0.052)	Loss 1.746 (1.882)	Acc@1 55.86 (54.01)	Acc@5 80.86 (79.64)
Test: [70/79]	Time 0.055 (0.051)	Loss 1.907 (1.884)	Acc@1 55.86 (53.93)	Acc@5 78.52 (79.52)
 * Acc@1 53.800 Acc@5 79.565
Accuracy of initialized INT8 model: 53.800
time: 3.98 s (started: 2021-08-07 15:28:52 +03:00)


# Fine-tune the compressed model

At this step, a regular fine-tuning process is applied to restore accuracy drop. Normally, it is required several epochs of tuning with a small learning rate, the same that is usually used at the end of the training of the original model. No other changes in the training pipeline are required. Here is the simple example.

In [16]:
# train for one epoch with NNCF
train(train_loader, model, criterion, optimizer, epoch=epochs)

# evaluate on validation set after Quantization-Aware Training (QAT case)
acc1 = validate(val_loader, model, criterion)

print(f'Accuracy of tuned INT8 model: {acc1:.3f}')

Epoch:[4][  0/313]	Time 0.360 (0.360)	Loss 0.973 (0.973)	Acc@1 78.91 (78.91)	Acc@5 90.23 (90.23)
Epoch:[4][ 50/313]	Time 0.083 (0.093)	Loss 0.972 (0.938)	Acc@1 78.91 (77.26)	Acc@5 90.62 (92.92)
Epoch:[4][100/313]	Time 0.083 (0.090)	Loss 1.131 (0.951)	Acc@1 69.53 (76.93)	Acc@5 90.62 (92.74)
Epoch:[4][150/313]	Time 0.089 (0.089)	Loss 0.940 (0.953)	Acc@1 75.78 (76.74)	Acc@5 92.97 (92.69)
Epoch:[4][200/313]	Time 0.081 (0.089)	Loss 0.911 (0.958)	Acc@1 77.34 (76.63)	Acc@5 91.80 (92.55)
Epoch:[4][250/313]	Time 0.087 (0.089)	Loss 0.994 (0.959)	Acc@1 76.56 (76.47)	Acc@5 91.41 (92.47)
Epoch:[4][300/313]	Time 0.085 (0.089)	Loss 0.909 (0.963)	Acc@1 77.34 (76.27)	Acc@5 92.58 (92.42)
Test: [ 0/79]	Time 0.359 (0.359)	Loss 2.133 (2.133)	Acc@1 50.78 (50.78)	Acc@5 76.56 (76.56)
Test: [10/79]	Time 0.040 (0.074)	Loss 1.627 (1.924)	Acc@1 61.72 (53.27)	Acc@5 82.42 (78.98)
Test: [20/79]	Time 0.051 (0.060)	Loss 1.817 (1.895)	Acc@1 53.91 (54.15)	Acc@5 84.38 (79.65)
Test: [30/79]	Time 0.038 (0.055)	Loss 1.986 (

# Export INT8 model to ONNX

In [17]:
if not int8_onnx_path.exists():
    # Export INT8 model to ONNX that is supported by the OpenVINO™ toolkit
    compression_ctrl.export_model(int8_onnx_path)
    print(f"INT8 ONNX model exported to {int8_onnx_path}.")

/home/nlyalyus/Developer/env/openvino_env/lib/python3.8/site-packages/torch/onnx/utils.py:305: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn("It is recommended that constant folding be turned off ('do_constant_folding=False') "
/home/nlyalyus/Developer/env/openvino_env/lib/python3.8/site-packages/nncf/torch/dynamic_graph/trace_tensor.py:36: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in th

INT8 ONNX model exported to output/resnet18_int8.onnx.
time: 853 ms (started: 2021-08-07 15:29:28 +03:00)


# Export ONNX models to OpenVINO™ Intermediate Representation (IR)

Call the OpenVINO Model Optimizer tool to convert the ONNX model to OpenVINO IR, with FP16 precision. The models are saved to the current directory. We add the mean values to the model and scale the output with the standard deviation by --mean_values and --scale_values arguments. It is not necessary to normalize input data before propagating it through the network with these options.

See the [Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer.

Executing this command may take a while. There may be some errors or warnings in the output. Model Optimization succesfully export to IR if the last lines of the output include: `[ SUCCESS ] Generated IR version 10 model`

In [18]:
if not fp32_ir_path.exists():
    !mo --input_model $fp32_onnx_path --input_shape "[1,3, $image_size, $image_size]" --mean_values "[123.675, 116.28 , 103.53]" --scale_values "[58.395, 57.12 , 57.375]" --data_type FP16 --output_dir $OUTPUT_DIR

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/nlyalyus/Developer/projects/openvino_notebooks/notebooks/302-pytorch-quantization-aware-training/output/resnet18_fp32.onnx
	- Path for generated IR: 	/home/nlyalyus/Developer/projects/openvino_notebooks/notebooks/302-pytorch-quantization-aware-training/output
	- IR output name: 	resnet18_fp32
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,3, 64, 64]
	- Mean values: 	[123.675, 116.28 , 103.53]
	- Scale values: 	[58.395, 57.12 , 57.375]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
	- Inference Engine found in: 	/home/nlyalyus/Developer/env/openvino_env/lib/pyth

In [19]:
if not int8_ir_path.exists():
    !mo --input_model $int8_onnx_path --input_shape "[1,3, $image_size, $image_size]" --mean_values "[123.675, 116.28 , 103.53]" --scale_values "[58.395, 57.12 , 57.375]" --data_type FP16 --output_dir $OUTPUT_DIR

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/nlyalyus/Developer/projects/openvino_notebooks/notebooks/302-pytorch-quantization-aware-training/output/resnet18_int8.onnx
	- Path for generated IR: 	/home/nlyalyus/Developer/projects/openvino_notebooks/notebooks/302-pytorch-quantization-aware-training/output
	- IR output name: 	resnet18_int8
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,3, 64, 64]
	- Mean values: 	[123.675, 116.28 , 103.53]
	- Scale values: 	[58.395, 57.12 , 57.375]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
	- Inference Engine found in: 	/home/nlyalyus/Developer/env/openvino_env/lib/pyth

# Benchmark model performance by computing inference time

Finally, we will measure the inference performance of the FP32 and INT8 models. To do this, we use [Benchmark Tool](https://docs.openvinotoolkit.org/latest/openvino_inference_engine_tools_benchmark_tool_README.html) - OpenVINO's inference performance measurement tool. By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput (frames per second) values.

> **NOTE**: In this notebook we run benchmark_app for 15 seconds to give a quick indication of performance. For more accurate performance, we recommended running benchmark_app in a terminal/command prompt after closing other applications. Run benchmark_app -m model.xml -d CPU to benchmark async inference on CPU for one minute. Change CPU to GPU to benchmark on GPU. Run benchmark_app --help to see an overview of all command line options.


In [ ]:
def parse_benchmark_output(benchmark_output):
    parsed_output = [line for line in benchmark_output if not (line.startswith(r"[") or line.startswith("  ") or line=="")]
    print(*parsed_output, sep='\n')

print(f'Benchmark FP32 model (IR)')
benchmark_output = ! benchmark_app -m $fp32_ir_path -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

print(f'Benchmark INT8 model (IR)')
benchmark_output = ! benchmark_app -m $int8_ir_path -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

Benchmark FP32 model (IR)
Count:      41544 iterations
Duration:   15002.82 ms
Latency:    2.06 ms
Throughput: 2769.08 FPS
Benchmark INT8 model (IR)


Show CPU Information for reference

In [ ]:
try:
    import cpuinfo
    print(cpuinfo.get_cpu_info()["brand_raw"])
except Exception:
    # OpenVINO installs cpuinfo, but if a different version is installed
    # the command above may not work
    import platform
    print(platform.processor())